In [5]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination
import sys, os
sys.path.append("./..")
import classes.powerdf as pdf
import matplotlib.pyplot as plt
import classes.powerdf as pdf
import classes.utils as utils
import matplotlib.pyplot as plt
from classes.data_loader import smarthome_dataset
import copy
import networkx as nx

cur_path = os.path.abspath(os.getcwd())
dataset_path = os.path.join(cur_path,"cleaned_HomeC.csv")
print(dataset_path)
df = pd.read_csv(dataset_path, low_memory = False)
dataset = smarthome_dataset("cleaned_HomeC.csv" , dayahead = False)
print("dataset length", len(dataset))

# fill missing values in
df.fillna(value = 0)
train_df = df.iloc[:50000]
test_df = df.iloc[50000:51000]
df = df.iloc[:]
print(len(df))

/Users/meezy/Documents/Carnegie/10-708_F23/GRIDPredict/notebooks/cleaned_HomeC.csv
dataset length 503910
503910


In [ ]:
from IPython.display import display, HTML
display(df)
print(df["visibility"])

In [45]:
def print_cpd(cpd,variable):
    for ind,state in enumerate(cpd.state_names[variable]):
        print("Probability of {} being {}".format(variable,state), cpd.values[ind])

def cpd_sample(cpd, variable, samples=1):
    values = np.array(cpd.state_names[variable])
    probs = np.array(cpd.values)
    print("values", values)
    print("probs", probs)
    return np.random.choice(values,samples, True, probs)


# EXAMPLE
network_edges = [('temperature','gen_Sol')]
template_model = BayesianNetwork(network_edges)

# Create Bayesian Estimator object to learn the model parameters from the data
b_est = BayesianEstimator(template_model, train_df)
model_parameters = b_est.get_parameters(prior_type="BDeu",equivalent_sample_size=5,pseudo_counts=None,n_jobs=-1,weighted=False)

cpds = model_parameters
learned_model, _ = utils.create_network_from_edges(network_edges, cpds, BayesianNetwork())

solver = VariableElimination(learned_model)
# Testing

errors = np.array([])
gt_values = np.array([])
for i,val in enumerate(range(50000,51000)):
    variable_values = []
    evidence_nodes = copy.copy(nodes)
    evidence_nodes.remove('gen_Sol')
    evidence_nodes = nodes
    for node in evidence_nodes:
        variable_values.append((node, df.iloc[i][node]))

    queried = False
    while (queried == False):
        evidence_dict = {}
        for ele in variable_values:
            evidence_dict[ele[0]] = ele[1]
        try:
            evidence_dict = {}
            for ele in variable_values:
                evidence_dict[ele[0]] = ele[1]
            cpd = solver.query(['gen_Sol'], evidence = evidence_dict)
            queried = True
            print("used evidence")
        except:
            variable_values.pop()
            if (variable_values == []):
                cpd = solver.query(variables=['gen_Sol'])
                queried = True

    queried_solar = float(cpd_sample(cpd, 'gen_Sol', samples=1)[0])
    gt_solar = float(df.iloc[i+10]['gen_Sol'])
    #print("queried solar", queried_solar)
    #print("gt solar", gt_solar)
    #print("error", queried_solar - gt_solar)
    errors = np.append(errors, queried_solar - gt_solar)
    gt_values = np.append(gt_values, gt_solar)

RMSE = (np.sqrt(np.mean(np.square((errors) / gt_values)))) * 100
MAE = (np.sum(np.abs(errors) / gt_values) * 100) / len(gt_values)

print("errors", errors)
print("mean error", np.mean(errors))
print("RMSE error", RMSE)
print("RMSE percent of average value ", 100 * RMSE / np.mean(gt_values))
print("MAE percent error", MAE)

nodes ['temperature', 'gen_Sol']
[<TabularCPD representing P(temperature:762) at 0x2bf9b9190>, <TabularCPD representing P(gen_Sol:11445 | temperature:762) at 0x2ba2c8c10>]
values [0.0000e+00 1.6700e-05 3.3300e-05 ... 5.3440e-01 5.4610e-01 5.5495e-01]
probs [1.59992738e-04 1.79990738e-04 1.59992738e-04 ... 2.00067368e-05
 2.00067368e-05 2.00067368e-05]
values [0.0000e+00 1.6700e-05 3.3300e-05 ... 5.3440e-01 5.4610e-01 5.5495e-01]
probs [1.59992738e-04 1.79990738e-04 1.59992738e-04 ... 2.00067368e-05
 2.00067368e-05 2.00067368e-05]
values [0.0000e+00 1.6700e-05 3.3300e-05 ... 5.3440e-01 5.4610e-01 5.5495e-01]
probs [1.59992738e-04 1.79990738e-04 1.59992738e-04 ... 2.00067368e-05
 2.00067368e-05 2.00067368e-05]
values [0.0000e+00 1.6700e-05 3.3300e-05 ... 5.3440e-01 5.4610e-01 5.5495e-01]
probs [1.59992738e-04 1.79990738e-04 1.59992738e-04 ... 2.00067368e-05
 2.00067368e-05 2.00067368e-05]
values [0.0000e+00 1.6700e-05 3.3300e-05 ... 5.3440e-01 5.4610e-01 5.5495e-01]
probs [1.59992738e-04

In [11]:
def print_cpd(cpd,variable):
    for ind,state in enumerate(cpd.state_names[variable]):
        print("Probability of {} being {}".format(variable,state), cpd.values[ind])

def cpd_sample(cpd, variable, samples=1):
    values = np.array(cpd.state_names[variable])
    probs = np.array(cpd.values)
    print("values", values)
    print("probs", probs)
    return np.random.choice(values,samples, True, p = probs)


# EXAMPLE

og_template_model = BayesianNetwork([('visibility', 'pressure'), ('visibility', 'Microwave'), ('pressure', 'temperature'), ('pressure', 'Fridge'), ('pressure', 'Furnace'), ('pressure', 'Garage door'), ('pressure', 'Dishwasher'), ('pressure', 'Barn'), ('temperature', 'Well'), ('temperature', 'gen_Sol'), ('windSpeed', 'Wine cellar'), ('humidity', 'use_HO'), ('dewPoint', 'humidity'), ('dewPoint', 'windSpeed'), ('precipIntensity', 'Living room')])
# 1000 data points
og_network_edges = [('visibility', 'pressure'), ('visibility', 'Microwave'), ('pressure', 'temperature'), ('pressure', 'Fridge'), ('pressure', 'Furnace'), ('pressure', 'Garage door'), ('pressure', 'Dishwasher'), ('pressure', 'Barn'), ('temperature', 'Well'), ('temperature', 'gen_Sol'), ('windSpeed', 'Wine cellar'), ('humidity', 'use_HO'), ('dewPoint', 'humidity'), ('dewPoint', 'windSpeed'), ('precipIntensity', 'Living room')]
# 100000 data points
#og_network_edges = [('cloudCover', 'dewPoint'), ('cloudCover', 'visibility'), ('cloudCover', 'humidity'), ('cloudCover', 'precipIntensity'), ('cloudCover', 'Garage door'), ('visibility', 'Barn'), ('humidity', 'windSpeed'), ('humidity', 'Furnace'), ('humidity', 'Microwave'), ('humidity', 'Kitchen'), ('temperature', 'Home office'), ('temperature', 'Dishwasher'), ('pressure', 'temperature'), ('Barn', 'gen_Sol')]

network_edges = utils.get_component_containing(nx.DiGraph(og_network_edges), "gen_Sol")
print("weather isolated edges", network_edges)
template_model = BayesianNetwork(network_edges)

#display(df)
#print(df["visibility"])

# Create Bayesian Estimator object to learn the model parameters from the data
b_est = BayesianEstimator(template_model, train_df)
model_parameters = b_est.get_parameters(prior_type="BDeu",equivalent_sample_size=5,pseudo_counts=None,n_jobs=-1,weighted=False)

cpds = model_parameters
learned_model, nodes = utils.create_network_from_edges(network_edges, cpds, BayesianNetwork())

solver = VariableElimination(learned_model)

# Testing

errors = np.array([])
gt_values = np.array([])
for i,val in enumerate(range(50000,51000)):
    variable_values = []
    evidence_nodes = copy.copy(nodes)
    evidence_nodes.remove('gen_Sol')
    evidence_nodes = nodes
    for node in evidence_nodes:
        variable_values.append((node, df.iloc[i][node]))

    queried = False
    while (queried == False):
        evidence_dict = {}
        for ele in variable_values:
            evidence_dict[ele[0]] = ele[1]
        try:
            evidence_dict = {}
            for ele in variable_values:
                evidence_dict[ele[0]] = ele[1]
            cpd = solver.query(['gen_Sol'], evidence = evidence_dict)
            queried = True
            print("used evidence")
        except:
            variable_values.pop()
            if (variable_values == []):
                cpd = solver.query(variables=['gen_Sol'])
                queried = True

    
    queried_solar = float(cpd_sample(cpd, 'gen_Sol', samples=1)[0])
    gt_solar = float(df.iloc[i+10]['gen_Sol'])
    print("queried solar", queried_solar)
    #print("gt solar", gt_solar)
    #print("error", queried_solar - gt_solar)
    errors = np.append(errors, queried_solar - gt_solar)
    gt_values = np.append(gt_values, gt_solar)

RMSE = (np.sqrt(np.mean(np.square((errors) / gt_values)))) * 100
MAE = (np.sum(np.abs(errors) / gt_values) * 100) / len(gt_values)

print("errors", errors)
print("mean error", np.mean(errors))
print("RMSE error", RMSE)
print("RMSE percent of average value ", 100 * RMSE / np.mean(gt_values))
print("MAE percent error", MAE)

graph {'gen_Sol', 'Furnace', 'Microwave', 'Well', 'Dishwasher', 'pressure', 'visibility', 'Barn', 'Fridge', 'temperature', 'Garage door'}
vertex gen_Sol
weather isolated edges [('visibility', 'pressure'), ('visibility', 'Microwave'), ('pressure', 'temperature'), ('pressure', 'Fridge'), ('pressure', 'Furnace'), ('pressure', 'Garage door'), ('pressure', 'Dishwasher'), ('pressure', 'Barn'), ('temperature', 'Well'), ('temperature', 'gen_Sol')]
nodes ['visibility', 'pressure', 'Microwave', 'temperature', 'Fridge', 'Furnace', 'Garage door', 'Dishwasher', 'Barn', 'Well', 'gen_Sol']
[<TabularCPD representing P(visibility:181) at 0x28f873a90>, <TabularCPD representing P(pressure:754 | visibility:181) at 0x177c23510>, <TabularCPD representing P(Microwave:564 | visibility:181) at 0x28ac20910>, <TabularCPD representing P(temperature:762 | pressure:754) at 0x28ac203d0>, <TabularCPD representing P(Fridge:4566 | pressure:754) at 0x28ac20850>, <TabularCPD representing P(Furnace:28688 | pressure:754) a